### **STEP-1**. 준비 작업
AI contents 실행 (python main.py) 후에 등록 과정을 진행 할 수 있습니다. !! 

jupyter kernel 을 제작합니다. 
```bash
    conda activate {ENV-NAME}           ## python main.py 실행 용 가상환경
    pip install ipykernel        
    python -m ipykernel install --user --name {ENV-NAME} --display-name {IPYKERNEL-NAME}
```


----

### **STEP-2**. Solution 정보 입력 및 Infra 정보 입력

##### Solution information (dict) 작성 
- inference_only: 파이프라인이 학습 및 추론 과정을 지원하는 표시     
  - ex1: 학습 과 추론 모두 지원 --> False     
  - ex2: 추론만 지원 --> True    
  - ex3: 학습 만은 미지원 --> ~~['train']~~ 

- solution_update: 기존 솔루션을 업데이트 할지 결정
  - True: 업데이트 진행. 기존 name 과 동일한 이름 입력 (name 이 존재하지 않으면 에러)  
  - False: 신규 생성. 기존 name 과 다르게 입력 (name 이 존재하면 에러)

- solution_name: 솔루션 이름 
  - 주의사항: 스페이스, 특수 문자, 한글은 미지원 

- solution_type: AI Solution 이 공유되는 계정 범위. 'private', 'public' 중에 하나 선택.   
  - 'public' : AI-Advisor 가 설치될 때, built-in 되어 설치 할 solution 의미.   
    (SYSTEM MANAGER 만 등록 가능)
  - 'private': 과제 단위로 공유 될 solution. 

- description: 솔루션이 Edge Conductor 의 UI 로 표시 될 때, 설명 가이드 작성 
  - overview: 솔루션에 대한 전반적을 설명을 작성. (markdown 을 지원)
  - 고려사항: overview 외에 다양한 sub-title 지원 예정.

- contents_type: re-train, re-labeling 을 위한 설명 
  - support_labeling: re-labeling 할지 결정. True 일 경우,   
    Edge Conductor 에서 re-labeling 기능 활성화.   
  - inference_result_datatype: EdgeApp 에 inference 결과를 표시하는 방법으로 'table', 'image' 중에 하나 선택. 
    AI Contents 제작 시, output.csv, output.jpg 를 생성해 두어야 함
  - train_datatype: re-train 에 사용될 데이터 포멧 결정으로 'table', 'image' 중에 하나 선택. 
    AI Contents 제작 시, output.csv, output.jpg 를 생성해 두어야 함

- train_gpu: True, False 중에 선택. True 일 경우, gpu 용 train docker container 제작 
- train_gpu: True, False 중에 선택. True 일 경우, gpu 용 inference docker container 제작 
- inference_arm: True, False 중에 선택. True 일 경우, arm 용 inference docker container 제작


    

In [ ]:
#----------------------------------------#
#        AI Solution Spec 작성           #
#----------------------------------------#
solution_info ={
    'inference_only': False, # True, False
    'solution_update': False,
    'solution_name': 'tcr-v2-test',
    'solution_type': 'private',
    'contents_type': {
            'support_labeling': False,
            'inference_result_datatype': 'table', # 'image'
            'train_datatype': 'table', # 'image'
            'labeling_column_name': ''
    },
    'train_gpu': False, ## cpu, gpu
    'inference_gpu': False,
    "inference_arm": False,  # amd, arm  
}

##### AI Solution 이 동작될 Infra_setup 파일 load 하기
./setting/infra_setup.yaml 를 참조하여 infra 환경을 설정합니다. 

### **STEP-3**. AI Solution 등록

##### 3-1. Username & Password 를 입력 

> !! 접속이 실패되는 경우, AI Conductor 에게 계정 생성을 요청 하시기 바랍니다.   
> Contact Us: hyunsoo0802.lim@lge.com  (LGE AI빅데이터담당, AI Conductor 담당자)

> Login 방식이 {'LOGIN_METHOD': 'ldap'} 로 설정되어 있는 경우, LGEP 의 ID & PW 를 사용


In [ ]:
import getpass

username = input('Username: ')
password = getpass.getpass('Password: ')

print("Your ID : ", username)
print("Your PW : ", password.replace(password, '*' * len(password)))

##### 3-2. ALO 솔루션 등록 실행
아래와 같은 과정이 순차적으로 실행 됨. 진행 중 Error 가 발생하게 되면, STEP-2 설정을 변경 함.
> Contact US: sehyun.song@lge.com (LGE AI빅데이터 담당, ALO 담당자)

AI Solution 등록 과정 Process:  
> 1.  solution name 입력 
> 2.  description & wrangler code 를 solution_metadata 에 삽입
> 3.  contents_type 을 solution_metadata 에 삽입
> 4.  description 의 solution icon 을 solution_metadata 에 삽입
> 5.  train 용 user parameter 를 solution_metadata 에 삽입
> 6.  s3 에 train data & artifacts 업로드 
> 7.  train 용 ecr repository 생성 및 docker container 업로드 
> 8.  inference 용 user parameter 를 solution_metadata 에 삽입
> 9.  s3 에 inference data & artifacts 업로드 
> 10. inference 용 ecr repository 생성 및 docker container 업로드 
> 11. solution 등록 



In [ ]:
import sys

try:
    del sys.modules['src.solution_register'] 
    del sys.modules['src.constants'] 
except:
    pass
from src.solution_register import SolutionRegister

# infra = "./setting/infra_config.yaml"
# infra = "./setting/example_infra_config/infra_config.meerkat.yaml"
infra = "./setting/example_infra_config/infra_config.customer.yaml"
# infra = "./setting/example_infra_config/infra_config.aictest.yaml"
register = SolutionRegister(infra_setup=infra, solution_info=solution_info)

register.login(username, password)

In [ ]:
register.debugging = False  ## default: False (skip 항목: docker 생성, solution 등록)
register.skip_generation_docker = False  ## default: False
register.run(username=username, password=password)

##### 등록된 AI Solution 테스트 하기. (Manager 권한만 가능)
- 테스트에 사용된 stream, solution instance 를 삭제 가능

In [ ]:
register.run_train(
    status_period = 10, ## 몇 초 간격으로 학습 상태를 체크할 것인지 설정
    delete_solution= False,  ## solution 을 삭제할 것인지 설정
    )

##### 등록한 Solution List 를 확인하기 (Manager 권한만 가능)

In [ ]:
register.list_solution()

In [ ]:
register.list_solution_instance()

In [ ]:
register.list_stream()

##### Solution 및 solution_instance 를 삭제 하기  

In [ ]:
register.delete_stream()
register.delete_solution_instance()

In [ ]:
register.delete_solution()